In [ ]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [ ]:
!pip install -q transformers

In [ ]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TRAIN D

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_train_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/train/trainP.tsv", header=None, sep="\t")

df_train_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/train/text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_train_clinical_P)):
    df_2 = load_articles(df_train_clinical_P, i)
    df_train_P = df_train_P.append(df_2)
    count = count + 1


df_train_P = df_train_P.set_axis(np.array(range(0,len(df_train_clinical_P))))
df_train_P[1] = df_train_clinical_P[1]
df_train_P.columns = ['content', 'labels']
df_train_P.head(10)

train = df_train_P

DEV D

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_dev_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/dev/devP.tsv", header=None, sep="\t")

df_dev_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/dev/text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_dev_clinical_P)):
    df_2 = load_articles(df_dev_clinical_P, i)
    df_dev_P = df_dev_P.append(df_2)
    count = count + 1


df_dev_P = df_dev_P.set_axis(np.array(range(0,len(df_dev_clinical_P))))
df_dev_P[1] = df_dev_clinical_P[1]
df_dev_P.columns = ['content', 'labels']
df_dev_P.head(10)

dev = df_dev_P

TEST D

In [ ]:
from pathlib import Path
df_test_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/test/testP.tsv", header=None, sep="\t")

df_test_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/test/text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n", dtype='unicode', error_bad_lines=False)
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_test_clinical_P)):
    df_2 = load_articles(df_test_clinical_P, i)
    df_test_P = df_test_P.append(df_2)
    count = count + 1


df_test_P = df_test_P.set_axis(np.array(range(0,len(df_test_clinical_P))))
df_test_P[1] = df_test_clinical_P[1]
LABEL_COLUMNS = ['content', 'labels']
df_test_P.columns = LABEL_COLUMNS
df_test_P.head(10)

test = df_test_P

**MATRIX TRANSFORMATION**

In [ ]:
new_labels_train = []
new_labels_test = []
new_labels_dev = []
lista = ['content']
for i in range(len(df_train_P['labels'])):
    new_labels_train.append([df_train_P['labels'][i]])
    lista = lista + new_labels_train[i]
for i in range(len(df_test_P['labels'])):
    new_labels_test.append([df_test_P['labels'][i]])
    lista = lista + new_labels_test[i]
for i in range(len(df_dev_P['labels'])):
    new_labels_dev.append([df_dev_P['labels'][i]])
    lista = lista + new_labels_dev[i]

    
####### TRAIN #######
myvec = np.zeros((len(df_train_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf = pd.DataFrame(data = myvec, index=range(len(df_train_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf.columns = lista
auxdf = auxdf.loc[:,~auxdf.columns.duplicated()]

df_train_P1 = df_train_P
df_train_P = df_train_P.drop_duplicates('content', ignore_index = True)
auxdf['content'] = df_train_P['content']

for j in range(len(auxdf['content'])):
    for i in range(len(df_train_P1['content'])):
        label = df_train_P1['labels'][i]
        if auxdf['content'][j]==df_train_P1['content'][i]:
            label = df_train_P1['labels'][i]
            auxdf[label][j] = 1 
            
train_matrix = auxdf            


#### TEST #####
myvec1 = np.zeros((len(df_test_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf1 = pd.DataFrame(data = myvec1, index=range(len(df_test_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf1.columns = lista
auxdf1 = auxdf1.loc[:,~auxdf1.columns.duplicated()]

df_test_P1 = df_test_P
df_test_P = df_test_P.drop_duplicates('content', ignore_index = True)
auxdf1['content'] = df_test_P['content']

for j in range(len(auxdf1['content'])):
    for i in range(len(df_test_P1['content'])):
        label1 = df_test_P1['labels'][i]
        if auxdf1['content'][j]==df_test_P1['content'][i]:
            label1 = df_test_P1['labels'][i]
            auxdf1[label1][j] = 1 
            
test_matrix = auxdf1            

###### DEV ######
myvec3 = np.zeros((len(df_dev_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf3 = pd.DataFrame(data = myvec3, index=range(len(df_dev_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf3.columns = lista
auxdf3 = auxdf3.loc[:,~auxdf3.columns.duplicated()]

df_dev_P1 = df_dev_P
df_dev_P = df_dev_P.drop_duplicates('content', ignore_index = True)
auxdf3['content'] = df_dev_P['content']

for j in range(len(auxdf3['content'])):
    for i in range(len(df_dev_P1['content'])):
        label3 = df_dev_P1['labels'][i]
        if auxdf3['content'][j]==df_dev_P1['content'][i]:
            label = df_dev_P1['labels'][i]
            auxdf3[label3][j] = 1 
            
dev_matrix = auxdf3            


##### TRAIN MATRIX #####
df_train_clinical = pd.DataFrame()
# train_matrix = pd.concat([train_matrix, dev_matrix], ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                               content  ...  cp151zz
0    We describe the case of a 37-year-old man with...  ...      0.0
1    We report the case of a 29-year-old woman who ...  ...      0.0
2    A 36-year-old male, with no relevant past medi...  ...      0.0
3    A 42-year-old woman underwent liver transplant...  ...      0.0
4    A 65-year-old male presented with an infravesi...  ...      0.0
..                                                 ...  ...      ...
430  We describe the case of a 47-year-old female p...  ...      0.0
431  A 65-year-old male patient diagnosed with OI f...  ...      0.0
432  A 72-year-old male in HD program from December...  ...      0.0
433  A 33-year-old woman presented to the emergency...  ...      0.0
434  We report the case of a 62-year-old patient wh...  ...      0.0

[435 rows x 871 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                               content  ...  cp151zz
0    Patient of 70 years old, retired miner, withou...  ...      0.0
1    A seventeen-year-old male with no history of i...  ...      0.0
2    A 62-year-old female patient, obese, with diab...  ...      0.0
3    A 53-year-old man with no relevant past medica...  ...      0.0
4    Male patient, 40 years old, with a history of ...  ...      0.0
..                                                 ...  ...      ...
219  The case described in our study is that of a 7...  ...      0.0
220  22-year-old man weight, 172 cm.), active milit...  ...      0.0
221  A 53-year-old male, road veteran cyclist, with...  ...      0.0
222  A 34-year-old male presented with a two-week h...  ...      0.0
223  A 57-year-old male with a history of arterial ...  ...      0.0

[224 rows x 871 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                               content  ...  cp151zz
0    A 29-year-old woman with a history of duodenal...  ...      0.0
1    58-year-old male at the time of transplantatio...  ...      0.0
2    A 22-year-old male patient, with no history of...  ...      0.0
3    A 35-year-old patient was admitted to our emer...  ...      0.0
4    A 90-year-old patient came to his urologist in...  ...      0.0
..                                                 ...  ...      ...
217  A five-year-old male patient who after sufferi...  ...      0.0
218  This is a 16-year-old adolescent, basketball p...  ...      0.0
219  We describe the case of a 58-year-old woman.Sh...  ...      0.0
220  We report the case of a 64-year-old male with ...  ...      1.0
221  A 73-year-old woman with a history of hyperten...  ...      0.0

[222 rows x 871 columns]
                                               content  ...  cp151zz
0    We describe the case of a 37-year-old man with...  ...      0.0
1    We 

In [ ]:
train_matrix1 = train_matrix
test_matrix1 = test_matrix
dev_matrix1 = dev_matrix

In [ ]:
total = []
for i in range(train_matrix1.shape[1]):
    if i>0:
        total.append(sum(train_matrix1.iloc[:,i]))


total_matrix = pd.concat([train_matrix1, test_matrix1, dev_matrix1], ignore_index=True)
lista = total_matrix.columns
lista = lista.delete(0)

mat = pd.DataFrame(total, index = lista)
mat = mat.sort_values(0, ascending = False)

maxi = len(lista) # coger 10 o más apariciones. Usar print de total para ver hasta donde coger
#maxi = 46

coger = mat.iloc[:maxi]
coger = coger.index
coger = coger.insert(0,'content')

train_matrix = train_matrix[coger]
test_matrix = test_matrix[coger]
dev_matrix = dev_matrix[coger]
# print(train_matrix)
# print(test_matrix)

In [ ]:
!pip install texthero
!pip install tweet-preprocessor

In [ ]:
import re 
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob,Word
from nltk.corpus import words
nltk.download('words')
import texthero as hero
import re
from texthero import stopwords

from nltk.corpus import wordnet

import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import tensorflow as tf


def lemma_per_pos(sent):
    '''function to lemmatize according to part of speech tag'''
    tweet_tokenizer=TweetTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = [lemmatizer.lemmatize(w) for w in  tweet_tokenizer.tokenize(sent)]
    return " ".join(lemmatized_list)

def df_preprocessing(df,feature_col):
    '''
    Preprocessing of dataframe
    '''
    stop = set(stopwords.words('english'))
    df[feature_col]= (df[feature_col].pipe(hero.lowercase).
                      pipe(hero.remove_urls).
                      pipe(hero.remove_digits).
                      pipe(hero.remove_punctuation).
                      pipe(hero.remove_html_tags) )
    # lemmatization
#     df[feature_col]= [lemma_per_pos(sent) for sent in df[feature_col]]
    # df[col_name]= hero.remove_stopwords(df[col_name],custom_stopwords)
    return df

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
target_col= train_matrix.columns[1:]
feature_col= train_matrix.columns[0]

In [ ]:
with tf.device('/GPU:0'):
    proc_train_df= df_preprocessing(train_matrix,feature_col)

In [ ]:
proc_test_df = df_preprocessing(test_matrix,feature_col)

In [ ]:
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import AdamW, get_linear_schedule_with_warmup
#import tensorflow_addons as tfa


#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-uncased'):
  '''Function to create the tokenizer'''

  tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
  return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_col,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_col].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)


    return [input_ids,attention_mask]

def bert_model(pretrained_weights,max_len,learning_rate):
  '''BERT model creation with pretrained weights
  INPUT:
  pretrained_weights: Language model pretrained weights
  max_len: input length '''
  print('Model selected:', pretrained_weights)
  bert=TFAutoModel.from_pretrained(pretrained_weights)
  
  # This is must if you would like to train the layers of language models too.
  for layer in bert.layers:
      layer.trainable = True

  # parameter declaration
#   step = tf.Variable(0, trainable=False)
#   schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [2e-0, 2e-1, 1e-2])
#   # lr and wd can be a function or a tensor
#   lr = learning_rate * schedule(step)
#   wd = lambda:lr * schedule(step)
#   optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

  optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
#   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # declaring inputs, BERT take input_ids and attention_mask as input
  input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
  attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

  bert= bert(input_ids,attention_mask=attention_mask)
  x= bert[0][:,0,:]
  x=tf.keras.layers.Dropout(0.1)(x)
  x= tf.keras.layers.Dense(128)(x)
  x=tf.keras.layers.Dense(64)(x)
  x=tf.keras.layers.Dense(32)(x)

  output=tf.keras.layers.Dense(maxi,activation='sigmoid')(x)

  model=Model(inputs=[input_ids,attention_mask],outputs=[output])
  # compiling model 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE,name='binary_crossentropy'),
                metrics=['accuracy'])
  return model


In [ ]:
pretrained_weights='bert-base-uncased'
max_len=256
epochs=20
learning_rate=2e-5
batch_size=4

In [ ]:
tokenizer= create_tokenizer(pretrained_weights)

In [ ]:
x_train= data_tokenization(proc_train_df,feature_col,max_len,tokenizer)

In [ ]:
y_train= proc_train_df[target_col].values
y_train

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
bert=bert_model(pretrained_weights,max_len,learning_rate)
bert.summary()

Model selected: bert-base-uncased


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning:

"`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"



109/109 [==============================] - 40s 250ms/step - loss: 0.6508 - accuracy: 0.0000e+00
Epoch 2/20
109/109 [==============================] - 28s 258ms/step - loss: 0.5239 - accuracy: 0.0000e+00
Epoch 3/20
109/109 [==============================] - 28s 254ms/step - loss: 0.3421 - accuracy: 0.0000e+00
Epoch 4/20
109/109 [==============================] - 28s 253ms/step - loss: 0.1837 - accuracy: 0.0000e+00
Epoch 5/20
109/109 [==============================] - 28s 255ms/step - loss: 0.0878 - accuracy: 0.0000e+00
Epoch 6/20
109/109 [==============================] - 28s 255ms/step - loss: 0.0466 - accuracy: 0.0000e+00
Epoch 7/20
109/109 [==============================] - 28s 253ms/step - loss: 0.0342 - accuracy: 0.0000e+00
Epoch 8/20
109/109 [==============================] - 28s 254ms/step - loss: 0.0303 - accuracy: 0.0713
Epoch 9/20
109/109 [==============================] - 28s 254ms/step - loss: 0.0284 - accuracy: 0.1218
Epoch 10/20
109/109 [==============================] - 2

In [ ]:
x_test= data_tokenization(test_matrix,feature_col,max_len,tokenizer)
x_test

[array([[  101,  5776,  1997, ...,  2015,  2024,   102],
        [  101,  1037,  9171, ...,     0,     0,     0],
        [  101,  1037,  2095, ..., 25320,  2007,   102],
        ...,
        [  101,  1037,  2095, ..., 22260,  9386,   102],
        [  101,  1037,  2095, ...,  5970,  1996,   102],
        [  101,  1037,  2095, ...,  2659,  2067,   102]], dtype=int32),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int32)]

In [ ]:
preds= bert.predict(x_test)
submiss_df= pd.DataFrame(preds, columns= target_col)
submiss_df

,bw03zzz,bw40zzz,bw20,bw24,4a02x4z,bn20,b030,bw21,5a1d,bv44zzz,0djd8zz,3e0t3cz,bw00zzz,08j1xzz,08j0xzz,0dj68zz,bt43zzz,0bh1,0tjb8zz,10e0xzz,0ttb,0fb0,0djdxzz,4a00x4z,0bj08zz,0vt0,bt40zzz,4a07x0z,b020,bw25,0tbb,3e0436z,bt4jzzz,b246zzz,5a1d00z,0db6,bw2f,5a19,0b11,bw30,...,0jb6,0dbp,bd14yzz,0d9b,0bbk0zx,b522,d9072zz,bw29,09bk,bw38,cb12,0h0t,0hbt,bh30,0mq6,0psn,0rspxzz,0kx1,0kxf,0rg6,0wbc,0bbl3zx,0d9e,cb12yzz,0bj0,bf40zzz,0b113f4,0b21xfz,0dh5,bf14,b413,b412,b549zzz,bf37,0mtn,dw16,0btg,0b9g8zx,0bb88zx,cp151zz
0,0.076200,0.229761,0.073685,0.144548,0.031615,0.071795,0.063779,0.098059,0.028526,0.129807,0.040147,0.022982,0.119128,0.005942,0.020420,0.033661,0.081664,0.047366,0.166627,0.013017,0.150087,0.085331,0.078722,0.020115,0.006972,0.013246,0.024091,0.008833,0.030483,0.038475,0.090752,0.013724,0.047940,0.013854,0.008583,0.020934,0.082328,0.010288,0.028521,0.031150,...,0.000105,1.742600e-07,0.000145,0.001630,0.000819,0.001661,0.002292,0.006511,0.006323,0.000218,0.001150,0.001324,0.004382,0.017614,0.000158,0.002679,0.001665,0.000012,0.004411,0.000763,0.000520,0.001889,0.000583,0.000423,0.000938,0.000017,0.000662,0.000025,0.014788,0.001100,0.000326,2.756252e-08,0.000065,0.000005,0.002921,0.000127,0.002211,0.011917,0.000027,0.000066
1,0.090535,0.279031,0.054833,0.092445,0.062142,0.022985,0.012398,0.031075,0.039357,0.015635,0.067319,0.019685,0.046776,0.039728,0.018017,0.076910,0.052605,0.040923,0.027143,0.026782,0.018313,0.024324,0.024403,0.012701,0.017722,0.011872,0.018079,0.005607,0.009470,0.017144,0.006041,0.010255,0.008495,0.010611,0.006067,0.025101,0.015009,0.009043,0.010816,0.006677,...,0.000103,3.539725e-08,0.000069,0.000643,0.000547,0.000837,0.000474,0.003039,0.002688,0.000132,0.000477,0.001435,0.002360,0.001783,0.000122,0.000719,0.001945,0.000002,0.004377,0.000141,0.000207,0.000645,0.000162,0.000367,0.000211,0.000003,0.000985,0.000005,0.001506,0.001040,0.000291,6.027709e-09,0.000059,0.000007,0.000698,0.000011,0.000832,0.004295,0.000007,0.000031
2,0.091225,0.117028,0.054713,0.044690,0.035432,0.022777,0.029730,0.035451,0.038724,0.107767,0.030106,0.141853,0.072146,0.072756,0.071137,0.012262,0.043447,0.019521,0.019115,0.046987,0.011780,0.015166,0.007212,0.037940,0.013285,0.026742,0.010192,0.012351,0.024019,0.018059,0.015846,0.021692,0.042966,0.015938,0.010737,0.008319,0.012514,0.016775,0.010930,0.041152,...,0.000213,4.508391e-07,0.000150,0.001416,0.000603,0.001708,0.001756,0.004202,0.008063,0.000208,0.000689,0.006495,0.009900,0.006598,0.000092,0.003276,0.007233,0.000004,0.008843,0.000379,0.001219,0.000731,0.001286,0.000987,0.000213,0.000025,0.001975,0.000037,0.002930,0.000566,0.000269,3.965030e-08,0.000168,0.000037,0.002601,0.000040,0.003475,0.011163,0.000023,0.000062
3,0.043238,0.479471,0.225596,0.071669,0.072096,0.088833,0.181723,0.152862,0.131507,0.061086,0.057978,0.029348,0.062164,0.024498,0.054343,0.041163,0.061493,0.045649,0.021999,0.017119,0.054967,0.027260,0.022450,0.055671,0.006729,0.018821,0.013932,0.010509,0.018273,0.042539,0.045139,0.018259,0.027741,0.006059,0.013347,0.037429,0.035720,0.009124,0.025949,0.052905,...,0.000210,1.732749e-07,0.000113,0.000891,0.000360,0.002435,0.003338,0.002658,0.002079,0.000127,0.000557,0.004873,0.005316,0.007927,0.000274,0.002350,0.003320,0.000009,0.002957,0.001179,0.000394,0.000447,0.000766,0.001064,0.001212,0.000015,0.001770,0.000021,0.008385,0.001848,0.000294,3.467522e-08,0.000058,0.000009,0.002054,0.000070,0.002725,0.009685,0.000015,0.000035
4,0.081013,0.168856,0.063592,0.112886,0.066633,0.071933,0.066752,0.056352,0.034093,0.046116,0.045414,0.012894,0.059734,0.006967,0.013887,0.041575,0.049711,0.028925,0.061823,0.015076,0.086003,0.035710,0.048915,0.010296,0.008518,0.014074,0.022258,0.008343,0.016088,0.018294,0.033189,0.007192,0.011397,0.006588,0.007198,0.014366,0.039709,0.010717,0.014616,0.015553,...,0.000052,4.149693e-08,0.000047,0.000492,0.000460,0.000283,0.001129,0.002053,0.003061,0.000078,0.000442,0.000638,0.002128,0.006637,0.000114,0.000864,0.000624,0.000005,0.002052,0.000282,0.00

In [ ]:
outputs = submiss_df>0.5
f1_score_micro = metrics.f1_score(test_matrix.iloc[:,1:], outputs, average='micro')
f1_score_macro = metrics.f1_score(test_matrix.iloc[:,1:], outputs, average='macro')
ap = metrics.average_precision_score(test_matrix.iloc[:,1:], outputs, average='micro')
print(f1_score_micro)
print(f1_score_macro)
print(ap)

0.020247469066366704
0.0004240102171136653
0.013679302779132131


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.

